In [44]:
import pandas as pd
import numpy
from tqdm import tqdm
import requests, zipfile, io
from sklearn.preprocessing import scale
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

## READ DATA

In [41]:
dfs = []

for year in tqdm(range(2013, 2019)):
    r = requests.get('http://www.tennis-data.co.uk/{0}/{0}.zip'.format(year))
    z = zipfile.ZipFile(io.BytesIO(r.content))
    df = pd.read_excel(z.open('{0}.xlsx'.format(year)))
    df['year'] = year
    dfs.append(df)

df = pd.concat(dfs)

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:14<00:00,  2.31s/it]
C:\Users\Toby\Anaconda2\envs\mne\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


In [42]:
df.head()

,ATP,AvgL,AvgW,B365L,B365W,Best of,Comment,Court,Date,EXL,...,W1,W2,W3,W4,W5,WPts,WRank,Winner,Wsets,year
0,1,2.78,1.42,3.00,1.36,3,Completed,Outdoor,2012-12-31,2.65,...,6.0,6.0,NaN,NaN,NaN,1215.0,28.0,Mayer F.,2.0,2013
1,1,2.05,1.73,2.20,1.61,3,Completed,Outdoor,2012-12-31,2.00,...,6.0,2.0,6.0,NaN,NaN,927.0,41.0,Nieminen J.,2.0,2013
2,1,3.58,1.28,3.75,1.25,3,Completed,Outdoor,2012-12-31,3.75,...,7.0,6.0,NaN,NaN,NaN,1830.0,19.0,Nishikori K.,2.0,2013
3,1,7.76,1.08,9.00,1.07,3,Completed,Outdoor,2012-12-31,8.00,...,6.0,6.0,NaN,NaN,NaN,1070.0,36.0,Baghdatis M.,2.0,2013
4,1,1.85,1.88,1.80,1.90,3,Completed,Outdoor,2013-01-01,1.87,...,6.0,6.0,NaN,NaN,NaN,897.0,43.0,Istomin D.,2.0,2013


## Compute target variable

W and L are recoded as A and B on alternate rows

In [5]:
for c in ['WRank', 'B365W']:
    df.loc[range(0, len(df), 2), c.replace('W', 'A')] = df.loc[range(0, len(df), 2), c]
    df.loc[range(1, len(df), 2), c.replace('W', 'A')] = df.loc[range(1, len(df), 2), c.replace('W', 'L')]
    
    df.loc[range(0, len(df), 2), c.replace('W', 'B')] = df.loc[range(0, len(df), 2), c.replace('W', 'L')]
    df.loc[range(1, len(df), 2), c.replace('W', 'B')] = df.loc[range(1, len(df), 2), c]
    
    df.loc[range(0, len(df), 2), 'y'] = 0
    df.loc[range(1, len(df), 2), 'y'] = 1

This calculates the difference in rank and betting odds between player A and B

In [7]:
df['rank_diff'] = df['ARank'] - df['BRank']
df['betting_diff'] = df['B365A'] - df['B365B']

df = df.dropna(subset=['rank_diff', 'betting_diff', 'y'])

## MACHINE LEARNING

In [35]:
clf = LogisticRegression()
X = df[['betting_diff']]
X = scale(X)
y = df['y']
clf.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

This gets the cross validated accuracy score across 3 folds of CV

In [36]:
cross_val_score(clf, X, y)

array([ 0.67314286,  0.68306636,  0.67620137])